In [14]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_bvp
from scipy.optimize import root

from simulation import *
from plotting import *

from joblib import Parallel, delayed

In [53]:
def solve_optimal_euclidean(nbody_sol, masses, x0, v0, m0, xf, mf, **kwargs):
    y0 = np.concatenate([x0, v0, m0])
    
    n = len(masses)
    
    #todo make these params
    v_e = 2
    alpha = 1
    beta = 1
    tf_guess = 5
    t_steps = 100
    #G = 6.67430e-11
    
    def control_root(control, m, p):
        #m has shape (N,1,1)
        #p has shape (N, 5, 1)
        #Get u in the shape (N, 2, 1)
        u = control.reshape((-1,2,1))
        
        N = p.shape[0]
        #Create "batch" identity matrix of shape (N,2,2)
        I = np.stack([np.eye(2) for i in range(N)])
        
        #Should have shape (N, 2, 3)x(N, 3, 1) = (N, 2, 1)
        dfdu_p = np.concatenate([
            v_e/m *I, -u/np.linalg.norm(u)
        ], axis=2)@p[:, 2:, :]
        
        #Should have shape (N, 2, 1)
        dldu = 2*alpha*v_e*u/m + 2*beta*u

        #return shape (N*2)
        return (dfdu_p + dldu).flatten()
    
    def ode(t, y, tf):
        N = len(t)
        #Unpack y after adjusting dimensions for batch matrix operations
        #y shape (10,N) -> (N,10) ->(N, 10, 1)
        y = np.expand_dims(y.T,2)
        #x shape (N,2,1)
        x = y[:, :2, :]
        #v shape (N,2,1)
        v = y[:, 2:4, :]
        #m shape (N, 1,1)
        m = y[:, 4:5, :]
        #p shape (N, 5, 1)
        p = y[:, 5:, :]
        
        #Create "batch" identity matrix of shape (N,2,2)
        #passing this into control_root may speed up finding the root
        I = np.stack([np.eye(2) for i in range(N)])

        batch_control_guess = np.random.random((N,2))
        #Solve for control: u should have shape (N,2,1)
        u = root(control_root, batch_control_guess, args=(m,p)).x.reshape(N,2,1)
        
        #Find the acceleration and big_ugly
        grav_accel = np.zeros((N,2,1))
        big_ugly = np.zeros((N,2,2))
        
        #rs shape (6n, N) -> (N, 6n) -> (N, 6n, 1)
        rs = np.expand_dims(nbody_sol.sol(tf[0]*t).T,2)
        for i in range(n):
            r = rs[:,6*i: 6*i+2,:] #shape (N, 2, 1)
            rx = r - x #shape (N, 2, 1)
            dist_rx = np.linalg.norm(rx, axis=1, keepdims=True) #dist_rx shape (N,1,1)
            #if np.isclose(dist_rx, 0).any():
            #    print("i",i, dist_rx.squeeze(), x.squeeze())
            grav_accel += masses[i] * (rx)/dist_rx**3
        
            #we can't use np.outer() here for the outer product because it will flatten the input arrays
            #We want to do a batch outer product that uses batch matrix multiplication of the shapes
            #(N,2,1) and (N,1,2) returns a matrix of shape (N,2,2)
            rx_outer = rx@np.transpose(rx, (0,2,1))
            big_ugly += masses[i] * (-I + 3*rx_outer/dist_rx**2)/dist_rx**3
        
        u_accel = v_e*u/m #Shape (N, 2, 1)
        accel = u_accel + grav_accel #accel should have shape (N, 2, 1)
        
        #Costate evolution
        
        #costate shape = (N,5,5)@(N,5,1) = (N,5,1)
        costate = -np.block([
            [np.zeros((N,2,2)), I, np.zeros((N,2,1))],
            [big_ugly, np.zeros((N,2,2)), -u_accel/m],
            [np.zeros((N,1,5))]
        ])@p
        
        #Concatenate the arrays together on axis=1 and squeeze axis 2 (which is just size 1) to get
        #an array of shape (N,10). Finally we need to transpose this to match the input size (10, N)
        return tf[0]*np.concatenate([
            v, #x' = v
            accel,
            -np.linalg.norm(u, axis=1, keepdims=True),
            costate
        ], axis=1).squeeze().T
    
    #Used to find control in bc(...)
    control_guess = np.array([1,0])
    mf_arr = np.array([[[mf]]]) #shape (1,1,1)
    
    #Since there are 11 unknowns, we need 11 boundary conditions
    #unknown state variables, 5 unknown costate, and
    #1 unknown parameter (the final time t_f)
    def bc(ya, yb, tf):
        #reshape m and p for the vectorized control_root function
        #(I had to vectorize it for the ode() function)
        p = yb[5:].reshape(1,5,1)
        
        #Solve for control at t_f
        u = root(control_root, control_guess, args=(mf_arr,p)).x #shape (2,)
        
        #Hamiltonian at final time, H(t_f) = p dot f - L
        #find the acceleration due to control and gravity at t_f
        control_accel = v_e*u/mf
        
        grav_accel = np.zeros(2)
        rs = nbody_sol.sol(tf[0])
        for i in range(n):
            dist = rs[6*i:6*i+2] - yb[:2]
            dist3norm = np.linalg.norm(dist)**3
            grav_accel += (masses[i]*dist/dist3norm)
        
        u2 = np.dot(u,u)
        pf =  yb[5:].dot(np.concatenate([
            yb[2:4], #y_2 = x'
            control_accel + grav_accel,
            [-np.sqrt(u2)]
        ]))
        
        #Lagrangian at t_f
        L = 1 + (alpha*v_e/mf + beta) * u2
        
        return np.concatenate([
            ya[:5] - y0, #x(0) = 0, v(0) = x'(0) = v0, m(0) = m0
            #This condition comes from our Bolza cost functional
            #which places a cost on x(t_f)'s distance from our desired target
            yb[5:7] + 2*(yb[:2] - xf), #p(t_f) = - Dphi/Dy(t_f)
            yb[7:9], #x'(t_f) is free (we could do the condition above)
            [(yb[4] - mf), #m(t_f) = mf Fixed final mass
            (pf - L)], #H(t_f) = p(t_f) dot f(x(t_f), u(t_f), th(t_f)) - L(t_f)
        ])
    
    t = np.linspace(0, 1, t_steps)
    guess = np.random.random((10, t_steps))
    return solve_bvp(ode, bc, t, guess, p=[tf_guess], **kwargs)

In [54]:
def attempt_solve():
    # Set up initial conditions and parameters
    t0 = 0
    tf = 25
    init = np.array([1, 1, 0, # Position 1
                     -1, -1, 0, # Position 2
                     #0, 0, 0, # Position 3
                     .35, -.25, 0, # Velocity/Momentum 1
                     -.35, .25, 0, # Velocity/Momentum 2
                     #.3, -.3, 0, # Velocity 3 
                     ])

    # Solve the system
    sol = solve_ivp(gravity_acceleration, (t0, tf), init, t_eval= np.linspace(t0, tf, 10000), dense_output=True)
    
    masses = np.ones(2)
    x0 = np.zeros(2)
    v0 = np.array([0.3, -0.3])
    m0 = np.array([2])
    target = np.array([0.1, -0.1])
    m_f = 0.1

    sol2 = solve_optimal_euclidean(sol, masses, x0, v0, m0, target, m_f, verbose=2, max_nodes=100)
    return sol2

In [55]:
attempt_solve()

   Iteration    Max residual  Max BC residual  Total nodes    Nodes added  
       1          4.11e+03       1.28e+01          100           (198)     
Number of nodes is exceeded after iteration 1. 
Maximum relative residual: 4.11e+03 
Maximum boundary residual: 1.28e+01


       message: 'The maximum number of mesh nodes is exceeded.'
         niter: 1
             p: array([2.74407673])
 rms_residuals: array([  45.68366097,   95.75341556,   52.27203847,   60.60596106,
         58.51429328,   11.70864818,   31.00755007,   67.62828196,
         78.27174554,   25.73228653,   11.64742528,   94.32275164,
         21.22939997,   25.73924202,  133.44927349,   62.16428547,
         43.40434711,  423.57268698,   73.53222088, 1125.99894236,
        710.55660865,  387.57998157,  222.73119264,   43.12590121,
         52.57858176,   14.69027923,   47.19110016,    8.33676343,
         12.27308588,   11.6494986 ,  267.45723727,   98.56886109,
        290.57378062,   43.66717295,   46.57224941,    8.03300356,
         48.61084129,   26.07927599,   38.634079  ,   83.26843451,
         42.70737636,   36.2600824 ,   38.76671018,   59.58456677,
         75.67813498,   68.62629691,   52.10382289,   56.61478528,
         86.733631  ,   14.85161721,   92.82453355,   28.34582